## Extracting 2D pixel values of 68 landmarks by running SBR on avi files

The purpose of this page is to:
1. generate 2d frames from the avi files of tasks
2. run SBR model on the frames to extract 2D facial landmark positions (x, y values of 68 landmarks)
3. save the results of landmarks supporimposed on images and create a .gif from frames of video
4. save .csv of x and y values 

In [53]:
# Copyright (c) Facebook, Inc. and its affiliates.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#
from __future__ import division

import torch, argparse
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import numpy as np
import os
from datasets import GeneralDataset as Dataset
from xvision  import transforms, draw_image_by_points
from models   import obtain_model, remove_module_dict
from config_utils import load_configure
from face_alignment import api as face_alignment

import pandas as pd
# GOT ERROR: No module named 'face_alignment.detection.blazeface'
from face_alignment.detection.blazeface import blazeface_detector

# face detector used by Diego in FAN analysis
from face_alignment.detection.sfd import sfd_detector
import warnings
import subprocess 


In [10]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

In [24]:
def sbr_env(FT):
    
    warnings.filterwarnings("ignore", category=DeprecationWarning)
    warnings.filterwarnings("ignore", category=UserWarning)
    print("I am here2")
    parser = argparse.ArgumentParser(description='Evaluate a single image by the trained model', formatter_class=argparse.ArgumentDefaultsHelpFormatter)
    parser.add_argument('--image',            type=str,   help='The evaluation image path.')
    parser.add_argument('--model',            type=str,   help='The snapshot to the saved detector.')
    parser.add_argument('--face',  nargs='+', type=float, help='The coordinate [x1,y1,x2,y2] of a face')
    parser.add_argument('--save',             type=str,   help='The path to save the visualized results.')
    parser.add_argument('--cpu',     action='store_true', help='Use CPU or not.')

 
    #args = parser.parse_args("AAA --file /path/to/sequences.txt".split())
    args = parser.parse_args(args=[])
    
    #args = parser.parse_args()
    print(args)
    print("I am here4")
    args.cpu   = False
    if FT == True:
        args.model = '/home/deniz/Code/SBR/lib/models/vgg16-397923af.pth' #TODO
    else:
        args.model = '/home/deniz/Code/SBR/lib/models/sbr_model.pth' #TODO
    
    snapshot = args.model
    if args.cpu: 
        snapshot = torch.load(snapshot, map_location='cpu')
    else: 
        snapshot = torch.load(snapshot)
    param = snapshot['args']
    print(param)
    model_config = load_configure('/home/deniz/Code/SBR/configs/Detector.config', None) #TODO
    print(model_config)
    net = obtain_model(model_config, param.num_pts + 1)
    print(net)
    
    if not args.cpu: 
        net = net.cuda()
    
    try:
        weights = remove_module_dict(snapshot['detector'])
    except:
        weights = remove_module_dict(snapshot['state_dict'])
    
    net.load_state_dict(weights)
    
    return net, args, model_config, param, FT

In [25]:
def evaluate(args, net, model_config, param):
    normalize   = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                      std=[0.229, 0.224, 0.225])
  
    eval_transform  = transforms.Compose([transforms.PreCrop(param.pre_crop_expand), transforms.TrainScale2WH((param.crop_width, param.crop_height)), transforms.ToTensor(), normalize])
    dataset = Dataset(eval_transform, param.sigma, model_config.downsample, param.heatmap_type, param.data_indicator)
    dataset.reset(param.num_pts)

    [image, _, _, _, _, _, cropped_size], meta = dataset.prepare_input(args.image, args.face)
  
    with torch.no_grad():
        if args.cpu: inputs = image.unsqueeze(0)
        else: inputs = image.unsqueeze(0).cuda()
        batch_heatmaps, batch_locs, batch_scos = net(inputs)
  
    np_batch_locs, np_batch_scos, cropped_size = batch_locs.cpu().detach().numpy(), batch_scos.cpu().detach().numpy(), cropped_size.cpu().detach().numpy()
    locations, scores = np_batch_locs[0,:-1,:], np.expand_dims(np_batch_scos[0,:-1], -1)

    scale_h, scale_w = cropped_size[0] * 1. / inputs.size(-2) , cropped_size[1] * 1. / inputs.size(-1)

    locations[:, 0], locations[:, 1] = locations[:, 0] * scale_w + cropped_size[2], locations[:, 1] * scale_h + cropped_size[3]
    prediction = np.concatenate((locations, scores), axis=1).transpose(1,0)
  
    if args.save:
        resize = 640, 360
        image = draw_image_by_points(args.image, prediction, 2, (255, 0, 0), [0, 0, 1280, 720], resize)#args.face
        image.save(args.save)
        print ('save the visualization results into {:}'.format(args.save))
    else:
        pass
    
    return prediction


In [26]:
# SET THE MODEL ARGUMENTS AND LOAD UP THE FACE DETECTOR MODEL

# If saving visualizations, this will run @ about 3fps
# If not, should be closer to ~20 (both on GPU)

#FT = "finetuned". Set to false here
net, args, model_config, param, FT = sbr_env(FT = False)

# N.B. I used the FAN-provided blazeface detector for consistency in my experiments, 
# but you can use blazeface from the actual Python library, or whatever face detector you want

#################################################
# GOT ERROR 
#bfd = blazeface_detector.BlazeFaceDetector(device = 'cuda')
###################################################

# Face detector model used by Dieggo in the FAN analysis 
#face_detector_model = r"./models/s3fd-619a316812.pth"
#face_detection_net = sfd_detector.SFDDetector(device=device, path_to_detector=face_detector_model, verbose=False)

bfd = blazeface_detector.BlazeFaceDetector(device = 'cuda')



I am here
I am here2
I am here3
I am here2
Namespace(cpu=False, face=None, image=None, model=None, save=None)
I am here4
Namespace(arg_flip=False, batch_size=8, crop_height=256, crop_perturb_max=30, crop_width=256, data_indicator='300W-68', error_bar=None, eval_ilists=None, eval_once=False, eval_vlists=None, heatmap_type='gaussian', lk_config='./configs/lk.config', model_config='./configs/Detector.config', num_pts=68, opt_config='./configs/LK.SGD.config', pre_crop_expand=0.2, print_freq=100, rand_seed=35123, rotate_max=20, save_path=None, scale_eval=1.0, scale_max=1.1, scale_min=0.9, scale_prob=1.0, sigma=4.0, train_lists=None, workers=8)
Configure(arch='cpm_vgg16', stages=3, dilation=[1], pooling=[True, True, True], downsample=8, argmax=4, pretrained=[True])


Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /home/deniz/.cache/torch/hub/checkpoints/vgg16-397923af.pth



Initialize cpm-vgg16 with configure : Configure(arch='cpm_vgg16', stages=3, dilation=[1], pooling=[True, True, True], downsample=8, argmax=4, pretrained=[True])
VGG16_base
VGG16_base(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace

In [126]:
population = ['ALS', 'PLS', 'Kennedys', 'Post-stroke', 'Healthy_Control']

tasks = ['BBP_NORMAL', 'NSM_OPEN', 'NSM_SPREAD', 'NSM_BIGSMILE' , 'RST_REST', 'NSM_KISS', 'NSM_PUCKER', 'DDK_PA']


base_path = '/media/deniz/easystore/facial_tracking_data_2021'

sub_folders_1 = '/02/RSSDK'
sub_folders_2 = '/02/LANDMARKS/Deniz_SBR'

In [127]:
master_avi_data = []
master_sbr_results = []
for p in population:
    data_dir = os.path.join(base_path, p) 
    print('working on pop: ', p)
    print(data_dir)
    if os.path.isdir(data_dir):
        for dirs in os.listdir(data_dir):
            task_dir = os.path.join(data_dir, dirs) + sub_folders_1
            result_dir = os.path.join(data_dir, dirs) + sub_folders_2
            if os.path.exists(task_dir):
                rawdata_list = os.listdir(task_dir)
                
                present = [os.path.join(task_dir, i) for i in rawdata_list if any(j in i for j in tasks) and i.endswith('color.avi')]
                master_avi_data=master_avi_data+ present
                master_sbr_results=master_sbr_results+ [result_dir] * len(present)

working on pop:  ALS
/media/deniz/easystore/facial_tracking_data_2021/ALS
working on pop:  PLS
/media/deniz/easystore/facial_tracking_data_2021/PLS
working on pop:  Kennedys
/media/deniz/easystore/facial_tracking_data_2021/Kennedys
working on pop:  Post-stroke
/media/deniz/easystore/facial_tracking_data_2021/Post-stroke
working on pop:  Healthy_Control
/media/deniz/easystore/facial_tracking_data_2021/Healthy_Control


In [137]:
# EXTRACT FRAMES FROM ALL 2D VIDEO FRAMES

for avi in master_avi_data:

    task_name = os.path.basename(avi)[:-4]
    tast_name_folder = task_name + '_raw_frames'
    result_dir = master_sbr_results[master_avi_data.index(avi)]
    result_task_frames = os.path.join(result_dir,tast_name_folder)

    if not os.path.exists(result_task_frames):
        os.makedirs(result_task_frames)
    # '/home/deniz/Code/SBR/movie_files/frames/%d.png'    
    subprocess.run(['ffmpeg', '-r', '1', '-i', f'{avi}', '-r', '1', os.path.join(result_task_frames, '%d.png')], shell = False) # TODO: fix directories as you want

    

In [155]:
master_sbr_results

['/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF02_SBK_20170517/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easystore/facial_tracking_data_2021/ALS/AF02_SBK_20170517/02/LANDMARKS/Deniz_SBR',
 '/media/deniz/easys

In [157]:
for i, f in enumerate(master_sbr_results):
    task_name = os.path.basename(master_avi_data[i])[:-4]
    folder_name_result = os.path.dirname(task_name)
    tast_name_folder = task_name + '_raw_frames'
    task_name_result = task_name + '_sbr_frames'
    print(f)
    print(task_name, folder_name_result, tast_name_folder, task_name_result)
    if i == 1:
        break

/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR
AF01_02_20170509_BBP_NORMAL_color  AF01_02_20170509_BBP_NORMAL_color_raw_frames AF01_02_20170509_BBP_NORMAL_color_sbr_frames
/media/deniz/easystore/facial_tracking_data_2021/ALS/AF01_SBK_20170509/02/LANDMARKS/Deniz_SBR
AF01_02_20170509_DDK_PATAKA_color  AF01_02_20170509_DDK_PATAKA_color_raw_frames AF01_02_20170509_DDK_PATAKA_color_sbr_frames


In [166]:
master_avi_data_main = master_avi_data
master_sbr_results_main = master_sbr_results

In [173]:
master_avi_data = master_avi_data[45:]
master_sbr_results = master_sbr_results[45:]

In [172]:
master_avi_data[44]

'/media/deniz/easystore/facial_tracking_data_2021/ALS/AF11_SBK_20170928/02/RSSDK/AF11_02_20170928_BBP_NORMAL_color.avi'

In [ ]:
# access the frames and process SBR on the data and store the results 

for i, f in enumerate(master_sbr_results):
    
    main_folder = f
    task_name = os.path.basename(master_avi_data[i])[:-4]
    folder_name_result = os.path.dirname(task_name)
    tast_name_folder = task_name + '_raw_frames'master_avi_data[30:]
    task_name_result = task_name + '_sbr_frames'
    full_sbr_path = os.path.join(f, task_name_result)
    full_raw_folder = os.path.join(f, tast_name_folder)
    if not os.path.exists(full_sbr_path):
        os.makedirs(full_sbr_path)
    fr = np.sort(np.asarray([int(i.split(sep = '.')[0]) for i in os.listdir(full_raw_folder)]))
    
    
    
    df_cols_p1 = []
    for i in range(0, 68):
            num = str(i)
            xx = 'landmark_' + num
            df_cols_p1.append(xx + '_x')
            df_cols_p1.append(xx + '_y')
            df_cols_p1.append(xx + '_z')
        
    header = df_cols_p1
    DF_3dpositions = pd.DataFrame(columns=header)

    # going through the frames 
    for f in fr:
        args.image  = os.path.join(full_raw_folder, str(f) + '.png') #TODO: a path to a target image
        faces       = bfd.detect_from_image(args.image)

        # the below face detector is used for FAN
        #faces       = face_detection_net.detect_from_image(args.image)
        box         = [faces[0][0], faces[0][1], faces[0][2], faces[0][3]]
        args.face   = box

        # args.save = [] ALTERNATIVE HYUJN
        args.save   = os.path.join(full_sbr_path, str(f) + '.png') # TODO: save location if you want to save images
        try:
            out = evaluate(args = args, net = net, model_config = model_config, param = param)
            # making it in the format compatable with my code 1x(68*3) landmarks from 0-67, x,y,z 
            out = out.T.reshape(1,-1)

        except:
            pass
        
        DF_3dpositions = DF_3dpositions.append(pd.Series(out.reshape((out.shape[1],)),index=DF_3dpositions.columns), ignore_index = True)
    
    DF_3dpositions.insert(loc=0, column='Video_Frame_number', value=fr)

    #base = "/home/deniz/Code/SBR/movie_files/results/"
    #name= "AF11_02_DDK_PATAKA_1305"

    landmark_file = os.path.join(main_folder,task_name+'_SBR_3D_landmarks.csv')
    DF_3dpositions.to_csv(landmark_file)

    ## Clean out the frames to save memory
    # for f in os.listdir('E:/Dropbox/frame/'): #TODO
    #     os.remove('E:/Dropbox/frame/' + f)

    ##  Convert your PNGs into an animated gif if you want
    ## Make sure the output file doesn't already exist; if it does, Python might crash
    output_gif = os.path.join(main_folder,task_name + '.gif')
    subprocess.run(['ffmpeg', '-i', os.path.join(full_sbr_path,'%d.png'),output_gif], shell = False) # TODO set your directories




# bELOW IS THE TESTING SCRIPT FOR A SINGLE VIDEO

In [33]:
## Subprocess is used to split a given video into frames. Optional, but
## HIGHLY recommended if you have MP4 files - they run through the model
## about 60% slower than PNG frames
## This isn't a problem with AVI files

## Also, if you use this, I recommend clearing the frame from memory immediately
## using the commented-out code @ line 131-132

vid_path    = r'/home/deniz/Code/SBR/movie_files/AF11_02_DDK_PATAKA_1305_color.avi'
       
subprocess.run(['ffmpeg', '-r', '1', '-i', f'{vid_path}', '-r', '1', '/home/deniz/Code/SBR/movie_files/frames/%d.png'], shell = False) # TODO: fix directories as you want



CompletedProcess(args=['ffmpeg', '-r', '1', '-i', '/home/deniz/Code/SBR/movie_files/AF11_02_DDK_PATAKA_1305_color.avi', '-r', '1', '/home/deniz/Code/SBR/movie_files/frames/%d.png'], returncode=0)

In [37]:
# Extract and serialize frame numbers - run if you used the above video processing stuff
fr = np.sort(np.asarray([int(i.split(sep = '.')[0]) for i in os.listdir('/home/deniz/Code/SBR/movie_files/frames')])) #TO DO



In [144]:
####  THE CASE OF TRYING THIS ON ONE VIDEO 

# making a datafarame to store the 3d LANDMARK RESULTS 
df_cols_p1 = []
for i in range(0, 68):
        num = str(i)
        xx = 'landmark_' + num
        df_cols_p1.append(xx + '_x')
        df_cols_p1.append(xx + '_y')
        df_cols_p1.append(xx + '_z')
        
header = df_cols_p1
DF_3dpositions = pd.DataFrame(columns=header)

# going through the frames 
for f in fr:
    args.image  = '/home/deniz/Code/SBR/movie_files/frames/' + str(f) + '.png' #TODO: a path to a target image
    faces       = bfd.detect_from_image(args.image)
    
    # the below face detector is used for FAN
    #faces       = face_detection_net.detect_from_image(args.image)
    box         = [faces[0][0], faces[0][1], faces[0][2], faces[0][3]]
    args.face   = box

    # args.save = [] ALTERNATIVE HYUJN
    args.save   = '/home/deniz/Code/SBR/movie_files/results/' + str(f) + '.png' # TODO: save location if you want to save images
                              
    out = evaluate(args = args, net = net, model_config = model_config, param = param)
    # making it in the format compatable with my code 1x(68*3) landmarks from 0-67, x,y,z 
    out = out.T.reshape(1,-1)
 
    DF_3dpositions = DF_3dpositions.append(pd.Series(out.reshape((out.shape[1],)),index=DF_3dpositions.columns), ignore_index = True)
    
DF_3dpositions.insert(loc=0, column='Video_Frame_number', value=fr)

base = "/home/deniz/Code/SBR/movie_files/results/"
name= "AF11_02_DDK_PATAKA_1305"

landmark_file = os.path.join(base,name+'_SBR_3D_landmarks.csv')
DF_3dpositions.to_csv(landmark_file)

## Clean out the frames to save memory
# for f in os.listdir('E:/Dropbox/frame/'): #TODO
#     os.remove('E:/Dropbox/frame/' + f)

##  Convert your PNGs into an animated gif if you want
## Make sure the output file doesn't already exist; if it does, Python might crash

output_gif = os.path.join(base,name + '.gif')
#test
#subprocess.run(['ffmpeg', '-i', f'{os.path.join(base+ '%d.png')}','output_gif], shell = False)
subprocess.run(['ffmpeg', '-i',os.path.join(base, '%d.png'), output_gif], shell = False)# TODO set your directories

#subprocess.run(['ffmpeg', '-i', f'/home/deniz/Code/SBR/movie_files/results/%d.png', '/home/deniz/Code/SBR/movie_files/output3.gif'], shell = False) # TODO set your directories


save the visualization results into /home/deniz/Code/SBR/movie_files/results/1.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/2.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/3.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/4.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/5.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/6.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/7.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/8.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/9.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/10.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/11.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/12.png
s

save the visualization results into /home/deniz/Code/SBR/movie_files/results/99.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/100.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/101.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/102.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/103.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/104.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/105.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/106.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/107.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/108.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/109.png
save the visualization results into /home/deniz/Code/SBR/movie_fil

save the visualization results into /home/deniz/Code/SBR/movie_files/results/196.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/197.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/198.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/199.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/200.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/201.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/202.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/203.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/204.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/205.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/206.png
save the visualization results into /home/deniz/Code/SBR/movie_fi

save the visualization results into /home/deniz/Code/SBR/movie_files/results/293.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/294.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/295.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/296.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/297.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/298.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/299.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/300.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/301.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/302.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/303.png
save the visualization results into /home/deniz/Code/SBR/movie_fi

save the visualization results into /home/deniz/Code/SBR/movie_files/results/390.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/391.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/392.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/393.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/394.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/395.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/396.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/397.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/398.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/399.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/400.png
save the visualization results into /home/deniz/Code/SBR/movie_fi

save the visualization results into /home/deniz/Code/SBR/movie_files/results/487.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/488.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/489.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/490.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/491.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/492.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/493.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/494.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/495.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/496.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/497.png
save the visualization results into /home/deniz/Code/SBR/movie_fi

save the visualization results into /home/deniz/Code/SBR/movie_files/results/584.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/585.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/586.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/587.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/588.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/589.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/590.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/591.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/592.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/593.png
save the visualization results into /home/deniz/Code/SBR/movie_files/results/594.png
save the visualization results into /home/deniz/Code/SBR/movie_fi

TypeError: expected str, bytes or os.PathLike object, not set

# Testing code for the pd in the format I want 

In [94]:
#df_cols_p1 = ["Video_Frame_number", 'Time_Stamp (s)']

df_cols_p1 = ["",""]
for i in range(0, 68):
        num = str(i)
        xx = 'landmark_' + num
        df_cols_p1.append(xx + '_x')
        df_cols_p1.append(xx + '_y')
        df_cols_p1.append(xx + '_z')
        


df_cols_p2 = ["",""]
for i in range(0, 68):
    df_cols_p2.append("x")
    df_cols_p2.append("y")
    df_cols_p2.append("z")

header = [np.array(df_cols_p1),
          np.array(df_cols_p2)]

DF_3dpositions = pd.DataFrame(columns=header)
#DF_3dpositions.loc[0]['Video_Frame_number'] = 0
DF_3dpositions

Empty DataFrame
Columns: [(, ), (, ), (landmark_0_x, x), (landmark_0_y, y), (landmark_0_z, z), (landmark_1_x, x), (landmark_1_y, y), (landmark_1_z, z), (landmark_2_x, x), (landmark_2_y, y), (landmark_2_z, z), (landmark_3_x, x), (landmark_3_y, y), (landmark_3_z, z), (landmark_4_x, x), (landmark_4_y, y), (landmark_4_z, z), (landmark_5_x, x), (landmark_5_y, y), (landmark_5_z, z), (landmark_6_x, x), (landmark_6_y, y), (landmark_6_z, z), (landmark_7_x, x), (landmark_7_y, y), (landmark_7_z, z), (landmark_8_x, x), (landmark_8_y, y), (landmark_8_z, z), (landmark_9_x, x), (landmark_9_y, y), (landmark_9_z, z), (landmark_10_x, x), (landmark_10_y, y), (landmark_10_z, z), (landmark_11_x, x), (landmark_11_y, y), (landmark_11_z, z), (landmark_12_x, x), (landmark_12_y, y), (landmark_12_z, z), (landmark_13_x, x), (landmark_13_y, y), (landmark_13_z, z), (landmark_14_x, x), (landmark_14_y, y), (landmark_14_z, z), (landmark_15_x, x), (landmark_15_y, y), (landmark_15_z, z), (landmark_16_x, x), (landmark_16_y, y), (landmark_16_z, z), (landmark_17_x, x), (landmark_17_y, y), (landmark_17_z, z), (landmark_18_x, x), (landmark_18_y, y), (landmark_18_z, z), (landmark_19_x, x), (landmark_19_y, y), (landmark_19_z, z), (landmark_20_x, x), (landmark_20_y, y), (landmark_20_z, z), (landmark_21_x, x), (landmark_21_y, y), (landmark_21_z, z), (landmark_22_x, x), (landmark_22_y, y), (landmark_22_z, z), (landmark_23_x, x), (landmark_23_y, y), (landmark_23_z, z), (landmark_24_x, x), (landmark_24_y, y), (landmark_24_z, z), (landmark_25_x, x), (landmark_25_y, y), (landmark_25_z, z), (landmark_26_x, x), (landmark_26_y, y), (landmark_26_z, z), (landmark_27_x, x), (landmark_27_y, y), (landmark_27_z, z), (landmark_28_x, x), (landmark_28_y, y), (landmark_28_z, z), (landmark_29_x, x), (landmark_29_y, y), (landmark_29_z, z), (landmark_30_x, x), (landmark_30_y, y), (landmark_30_z, z), (landmark_31_x, x), (landmark_31_y, y), (landmark_31_z, z), (landmark_32_x, x), (landmark_32_y, y), ...]
Index: []

[0 rows x 206 columns]